<a href="https://colab.research.google.com/github/miketreyu/data-science/blob/main/Sesion2_practica_sin_terminar(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a aprender a utilizar Pytorch con operaciones que nos demuestren las ventajas de su uso.

1.1. Crea un array de NumPy y un tensor de Pytorch en base a la siguiente lista de Python:

In [1]:
import random
import numpy as np
import torch

seed = 64
random.seed(seed)
lista = [[float(random.randint(0, 9)) for i in range(0, 5000)] for i in range(0, 5000)]
np_array = np.array(lista)
t_tensor = torch.tensor(np_array, dtype=torch.float32)



1.2. Completa la función "inverse_numpy". Esta función deberá recibir como parámetro un array de numpy de dos dimensiones, y devolver la matriz inversa correspondiente, utilizando numpy. Utilízala para invertir `np_array`.

In [2]:
def inverse_numpy(array: np.array) -> np.array:
    """
    Esta función recibe un array de numpy de dos dimensiones
    y devuelve la matriz inversa correspondiente.

    Parámetros:
    array (np.array): Matriz de entrada.

    Returns:
    np.array: Matriz inversa de la entrada.
    """
    return np.linalg.inv(array)

np_array_inv = inverse_numpy(np_array)



NameError: name 'np' is not defined

1.3. Completa la función "inverse_torch". Esta función deberá recibir como parámetro un tensor de torch de dos dimensiones y el dispositivo en el que ejecutar la operación, y devolver la matriz inversa correspondiente, utilizando torch. Utilízala para invertir `t_tensor`.

In [6]:
def inverse_torch(tensor: torch.tensor, device: str) -> torch.tensor:
    """
    Esta función recibe un tensor de PyTorch y devuelve la matriz inversa correspondiente.

    Parámetros:
    tensor (torch.Tensor): Matriz de entrada.
    device (str): Dispositivo en el que realizar el cálculo ('cpu' o 'cuda').

    Returns:
    torch.Tensor: Matriz inversa de la entrada.
    """
    # Verificar si el dispositivo es válido
    if device not in ['cpu', 'cuda']:
        raise ValueError("El dispositivo debe ser 'cpu' o 'cuda'.")

    # Mover el tensor al dispositivo especificado
    tensor = tensor.to(device)

    # Calcular la inversa utilizando PyTorch
    try:
        inv_tensor = torch.inverse(tensor)
    except RuntimeError as e:
        print(f"Error al calcular la inversa: {e}")
        return None

    return inv_tensor

# Calcular la inversa en la CPU
inv_tensor_cpu = inverse_torch(t_tensor, 'cpu')
print(inv_tensor_cpu)

# Si tienes una GPU disponible, también puedes calcular la inversa en la GPU
if torch.cuda.is_available():
    inv_tensor_cuda = inverse_torch(t_tensor, 'cuda')
    print(inv_tensor_cuda)

tensor([[ 9.4227e-03,  1.5325e-03, -5.9567e-04,  ..., -3.4978e-03,
          3.3604e-03, -5.7445e-03],
        [-2.0682e-05,  2.5677e-03,  3.0248e-03,  ..., -9.7142e-05,
          2.5897e-03, -1.7479e-03],
        [ 1.1557e-03,  9.0693e-04, -1.5543e-03,  ..., -2.2089e-04,
         -1.2010e-03, -4.1960e-04],
        ...,
        [-4.9247e-03, -2.3191e-03,  7.6789e-03,  ..., -9.3756e-04,
          4.3603e-03, -4.6544e-03],
        [-2.7706e-03,  2.4509e-03,  3.7010e-03,  ...,  8.3992e-04,
          3.1883e-03, -5.1375e-04],
        [ 7.6167e-04,  7.5150e-05, -6.1954e-03,  ...,  9.2874e-04,
         -1.0212e-03,  4.0348e-04]])


1.4. Utiliza la siguiente función para comparar cuánto tardan las funciones que has creado. Compara inverse_numpy contra inverse_torch en cpu contra inverse_torch en CUDA.

In [1]:
from timeit import default_timer as timer

def time_inverses(inv_function):
    start_time = timer()
    inv_function()
    end_time = timer()
    time_taken = end_time - start_time
    print(f"tiempo: {time_taken}")

inverse_numpy_l = lambda: inverse_numpy(t_tensor)
inverse_torch_cpu = lambda: inverse_torch(t_tensor, "cpu")
inverse_torch_gpu = lambda: inverse_torch(t_tensor, "cuda")

time_inverses(inverse_numpy_l)
time_inverses(inverse_torch_cpu)
time_inverses(inverse_torch_gpu)






NameError: name 'inverse_numpy' is not defined

# Entrenar un modelo multicapa

Vamos a utilizar un modelo multicapa para predecir sobre el dataset iris de sklearn. Primero, ejecuta la siguiente celda para conseguir la X y la y.

In [ ]:
# ----------- Ejecuta esta celda ----------- #

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X = iris['data']
y = iris['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=1./3, random_state=1)

torch.manual_seed(1)

X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)


2.1. Convierte la X_train_norm y la y_train en un [TensorDataset](https://pytorch.org/docs/stable/data.html?highlight=tensordataset#torch.utils.data.TensorDataset) de torch. Recuerda que primero debes convertir la X y la y en tensores. Después, introduce ese dataset en un [DataLoader](https://pytorch.org/docs/stable/data.html?highlight=dataloader#torch.utils.data.DataLoader) de torch, con batch_size de 2 y shuffle igual a True.

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader



2.2. Construye el modelo. Este debe tener dos capas [lineales y completamente conectadas](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html).
- Primero, construye cada capa dentro del método `__init__()` como atributos de la clase (recuerda que para ello, debes escribir `self` delante del nombre de la variable, como el `this` en Java).
- Después, en el método `forward`, aplica primero la operación de la capa y después la función de activación, para cada una de las dos capas. La función de activación de la capa uno deberá ser una [sigmoide](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html), y la de la capa 2, una [Softmax con dim=1](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html).

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        # La definición de las capas va aquí
        self.layer1 =
        self.layer2 =

    def forward(self, x):
        # El proceso a través de las capas va aquí
        return x

2.3. Define el tamaño de las capas. El tamaño de `input_size` viene dado por X. El de la capa oculta debe ser 16, y el de la capa de salida debe ser 3, ya que nuestra clase tiene tres valores posibles. Usa estos valores para construir un modelo con la clase `Model`.

In [ ]:
input_size = X_train_norm.shape[1]
hidden_size =
output_size =

model =

2.4. Define el learning_rate, la función de pérdida, y el optimizador.
- El learning_rate deberá ser 0.001.
- La función de pérdida deberá ser la de [entropía cruzada](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

In [ ]:
learning_rate =
loss_fn =

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

2.5. Ejecuta el siguiente código para entrenar el modelo con los parámetros que has definido. Míralo con detenimiento para comprender qué está pasando.

In [ ]:
num_epochs = 100
loss_hist = [0] * num_epochs
accuracy_hist = [0] * num_epochs

for epoch in range(num_epochs):

    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch.long())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_hist[epoch] += loss.item()*y_batch.size(0)
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist[epoch] += is_correct.sum()

    loss_hist[epoch] /= len(train_dl.dataset)
    accuracy_hist[epoch] /= len(train_dl.dataset)

2.6. Ejecuta esta celda para ver cómo progresa tu modelo a lo largo de cada época.

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 2, 1)
ax.plot(loss_hist, lw=3)
ax.set_title('Training loss', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(accuracy_hist, lw=3)
ax.set_title('Training accuracy', size=15)
ax.set_xlabel('Epoch', size=15)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()

#plt.savefig('figures/12_09.pdf')

plt.show()

2.7. Utiliza el método `predict` del modelo para predecir sobre `X_test`. Recuerda que primero debes normalizar la X. Puedes ver cómo hacerlo en el apartado 2.1. Una vez lo tengas, utiliza el método [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) para obtener una matriz de confusión y evaluar tu modelo.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

